In [ ]:
#import pandas as pd
#from transformers import BertTokenizer, BertModel
#import torch
#from bertopic import BERTopic
import pandas as pd
import nltk
from transformers import BertTokenizer, BertModel
from gensim import corpora, models
from gensim.models import ldamodel


In [ ]:
#count number of words in each predefined cateogry (combine w sentiment analysis -/+)
#categorize review based on count/sentiment analysis
#then do topic modeling,popular keyword count,named entity recognition on each of the categorized review categories 
#retrain and add the popular words back into predefined list
#categorize reviews based on keyword prevalence 

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')


In [ ]:
df=pd.read_csv('fritto.csv',index_col=0)
df=df.drop_duplicates(['Name','Date Posted'])
df

In [ ]:
df_bad_reviews=df[df['Ratings']<4]
#for i in df_bad_reviews['Reviews']:
    #print(i)

In [ ]:
#remove stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(review):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(review)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words and len(word) >= 3]
    return ' '.join(filtered_tokens)

df['processed_reviews'] = df['Reviews'].apply(preprocess_text)


In [ ]:
df

In [ ]:
#Define categories
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Sample restaurant reviews
reviews=df['processed_reviews'].to_list()

# Preprocessing steps
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
preprocessed_reviews = []

for review in reviews:
    tokens = word_tokenize(review.lower())  # Tokenization and lowercase
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum() and token not in stop_words]  # Lemmatization and removing stopwords
    preprocessed_reviews.append(" ".join(filtered_tokens))

# Count Vectorization
vectorizer = CountVectorizer(max_features=1000)  # Limit the number of features for demonstration
X = vectorizer.fit_transform(preprocessed_reviews)

# LDA Modeling
num_topics = 15  # Number of topics/themes
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Display topics and their top words
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-5 - 1:-1]]  # Display top 5 words per topic
    print(f"Topic {topic_idx + 1}: {' | '.join(top_words)}\n")


In [ ]:
#df['Assigned Topic'].value_counts()

In [ ]:
#word count x sentiment x similarity - no negative keyword emphasis - 1 (apply to negative keyword emphasis)
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from textblob import TextBlob  # For sentiment analysis
from sklearn.metrics.pairwise import cosine_similarity


# Preprocess the reviews
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum()]  # Remove non-alphanumeric characters
    return tokens

reviews=df['processed_reviews'].to_list()
processed_reviews = [preprocess_text(review) for review in reviews]

# Train Word2Vec model on the preprocessed reviews
word2vec_model = Word2Vec(processed_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Define the main words for each predefined topic along with additional related words (in lowercase)
predefined_topics = {
    "service": ['host','listen','listening','delivery','call','wait','right away','service','slow','time','menu','hostess','table','seat','host','waiter','table','receptionist','quick','responsive','accommodating','waitress','seated','rude','took','order','ordering',"service",'min','minutes','server', "excellent",'check', "staff", "attentive","friendly", "helpful",'courteous', "prompt",'mean','answer', "efficient", "knowledgeable",'phone','polite'],
    "ambiance": ["ambiance",'clean','dirty','bathrooms','decor','atmosphere','temperature','AC','tables','toilet','kitchen','party','place','family','casual','formal','romantic','spot','festive','lively','intimate','loud','noisy','bar','seats','chairs','location', "atmosphere", "decor", "cozy", "inviting", "warm", "stylish", "elegant", "relaxed",'parking','building','seating','park','lights'],
    "prices": ["prices",'price','total','bill was','check was','worth','value','money','pricey','pricy','portions','portion size','quantity', "reasonable",'over-priced','over','priced','loaded', "affordable",'inexpensive','wallet', "cost", "expensive", "cheap", "wallet-friendly","large portions","large plate"],
    "quality": ['quality','lemon','runny','lime','takeout','vegan','gnocchi','vegetarian','fresh','well-prepared','stale','well','prepared','authentic','atomic','calamari','pillows','puffs','cheese','ricotta','dish','dessert','inedible','flourless','tiramisu','satisfying','caprese','turkey','steak','chicken','sausage','lasagna','jumbo','like','pasta','salt','bread','shrimp','chocolate','cake','cream','drinks','wine', "delicious",'taste', "tasty", "flavorful",'runny','flavorless','flavor',"fresh",'disgusting','penne','fettuccine','alfredo', "high-quality",'prepared', "well-prepared",'garlic','bland','unflavorful','oily','oil','greasy','cream','heavy','sauce','fried','filling','sauces','ravioli'],
    "overall": ['overall','fritto misto','santa monica','la','los angeles','card','somewhere different','somewhere else','course','refund','reviews','review',"best spot",'food','come again','avoid','meal','menu','back','return','choice','elsewhere', "experience", "visit",'would not return', "would return",'come back','place','italian','restaurant','website','app', "would recommend to a friend", "would return", "would not return",'people','crowd','italian','authentic','waste'],
}

# Negative sentiment words
negative_words = [
    "bad", "horrible", "terrible", "poor", "disgusting",
    "awful", "unpleasant", "dreadful", "unappetizing",
    "disappointing", "mediocre", "unpalatable", "inferior",
    "overpriced", "unsatisfactory", "disastrous", "unimpressive",
    "bland", "gross", "repulsive", "underwhelming",
    "inedible", "unflavorful", "miserable", "regrettable",
    "subpar", "lousy", "offensive", "unenjoyable",
    "stale", "inedible", "undercooked", "soggy","horrendous"]

negative_word_weights = {
    "bad": 1.0,
    "horrible": 2.0,
    "Horrendous":2.0,
    "terrible": 2.0,
    "poor": 1.0,
    "disgusting": 2.0,
    "awful": 1.2,
    "unpleasant": 1.3,
    "dreadful": 1.5,
    "unappetizing": 1.2,
    "disappointing": 1.2,
    "mediocre": 1.1,
    "unpalatable": 1.2,
    "inferior": 1.0,
    "overpriced": 1.2,
    "unsatisfactory": 1.1,
    "disastrous": 1.5,
    "unimpressive": 1.1,
    "bland": 1.0,
    "gross": 1.2,
    "repulsive": 1.4,
    "underwhelming": 1.1,
    "inedible": 1.3,
    "unflavorful": 1.2,
    "miserable": 1.3,
    "regrettable": 1.2,
    "subpar": 1.1,
    "lousy": 1.2,
    "offensive": 1.3,
    "unenjoyable": 1.2,
    "stale": 1.1,
    "undercooked": 1.2,
    "soggy": 1.2,
}

# Transition words for sentence transitions
sentence_transitions = ['but', 'and', 'however', 'although', 'yet','so', 'nevertheless', 'while', 'though', 'or']


def calculate_sentiment(review):
    sentiment_scores = [TextBlob(word).sentiment.polarity for word in review]
    sentiment_distribution = [score - 1 for score in sentiment_scores]  # Shift to negative range for negative sentiment
    sentiment_distribution /= np.sum(np.abs(sentiment_distribution))  # Normalize
    sentiment = np.argmax(sentiment_distribution)

    # Adjust the sentiment score based on the order of the words and the context
    for i in range(len(sentiment_scores)):
        if sentiment_scores[i] > 0:
            sentiment_distribution[i] *= (-1) ** (i + 1)

    sentiment = np.argmax(sentiment_distribution)

    return sentiment

def calculate_sentiment_entropy(review):
    sentiment_scores = [TextBlob(word).sentiment.polarity for word in review]
    entropy = -sum(score * np.log2(score) for score in sentiment_scores if score != 0)
    return entropy

def get_sentiment(word):
    return TextBlob(word).sentiment.polarity

def get_main_topic(review):
    review_text = ' '.join(review)  # Convert preprocessed words back into a string
    sentiment_scores = [get_sentiment(word) for word in review]  # Define sentiment_scores here
    sentiment = calculate_sentiment(review_text)
    
    negative_sentiment_words = [word for word in review if word in negative_words]
    negative_sentiment_scores = [get_sentiment(word) for word in negative_sentiment_words]
    
    review_embedding = np.nanmean(
        [word2vec_model.wv[word] for word in review if word in word2vec_model.wv.key_to_index], axis=0
    )
    
    if np.any(np.isnan(review_embedding)):
        return None
    
    max_score = float('-inf')
    main_topic = None
    second_topic = None
    highest_negative_score = 0
    prioritized_categories = []
    
    for topic, words in predefined_topics.items():
        count = sum(1 for word in words if word in review)

        topic_embedding = np.nanmean(
            [word2vec_model.wv[word] for word in words if word in word2vec_model.wv.key_to_index], axis=0
        )

        if np.any(np.isnan(topic_embedding)):
            continue

        similarity = cosine_similarity([review_embedding], [topic_embedding])[0][0]

        # Prioritize topics containing negative sentiment keywords
        negative_sentiment_score = sum(1 for word in review if word in negative_words)
        if negative_sentiment_score > 0 and any(word in words for word in negative_words):
            combined_score = similarity + (0.5 * negative_sentiment_score) + (0.5 * count)
        else:
            combined_score = similarity + (0.5 * sentiment) + (0.5 * count)

        if combined_score > max_score:
            second_topic = main_topic
            max_score = combined_score
            main_topic = topic
            prioritized_categories = [topic]
        elif combined_score == max_score:
            second_topic = topic
    sentiment_entropy = calculate_sentiment_entropy(review)
    if sentiment_entropy > 1.0:
        return main_topic
    #f"{main_topic}, {second_topic}"
    else:
        return main_topic


# Assign each review to one of the predefined topics
review_topics = [get_main_topic(review) for review in processed_reviews]

# Print the assigned topics for each review along with scores and negative word information
for i, review in enumerate(reviews):
    main_topic = review_topics[i]
    count_negative_words = sum(1 for word in preprocess_text(review) if word in negative_words)
    negative_word_score = sum(negative_word_weights.get(word, 1.0) for word in preprocess_text(review) if word in negative_words)
    
    review_words = preprocess_text(review)
    review_embedding = np.nanmean(
        [word2vec_model.wv[word] for word in review_words if word in word2vec_model.wv.key_to_index], axis=0
    )
    
    sentiment_scores = [TextBlob(word).sentiment.polarity for word in review_words]
    sentiment = np.argmax([score + 1 for score in sentiment_scores])  # Shift to positive range
    review_data = {
        'Assigned Topic': main_topic,
        'Count of Negative Keywords': count_negative_words,
        'Aggregate Negative Score': negative_word_score
    }
    for category, words in predefined_topics.items():
        topic_embedding = np.nanmean(
            [word2vec_model.wv[word] for word in words if word in word2vec_model.wv.key_to_index], axis=0
        )
        
        if np.any(np.isnan(topic_embedding)):
            category_score = "N/A"
        else:
            similarity = cosine_similarity([review_embedding], [topic_embedding])[0][0]
            sentiment_score = (0.5 * sentiment) if any(word in words for word in review_words) else 0
            category_score = similarity + sentiment_score
            if category == 'service':
                review_data['Service Score'] = category_score
            elif category == 'ambiance':
                review_data['Ambiance Score'] = category_score
            elif category == 'prices':
                review_data['Prices Score'] = category_score
            elif category == 'quality':
                review_data['Quality Score'] = category_score
            elif category == 'overall':
                review_data['Overall Score'] = category_score
    #review_data['Overall Score'] = max(
        review_data.get('Overall Score', float('-inf')),
        review_data.get('Service Score', float('-inf')),
        review_data.get('Ambiance Score', float('-inf')),
        review_data.get('Prices Score', float('-inf')),
        review_data.get('Quality Score', float('-inf'))
        #)
    df.loc[df.index[i], review_data.keys()] = list(review_data.values())


In [ ]:
df

In [ ]:
df_copy=df.copy()

In [ ]:
#df

In [ ]:
df['Assigned Topic'].value_counts()

In [ ]:
import pandas as pd

# Sample DataFrame
data = {'Assigned Topic': ['quality', 'prices', 'overall', 'ambiance', 'service', 'quality']}
#df = pd.DataFrame(data)

# Initialize an empty dictionary to store the DataFrames
dataframes = {}

# List of keywords
keywords = ["quality", "prices", "overall", "ambiance", "service"]

# Create a DataFrame for each keyword
for keyword in keywords:
    # Use the keyword to filter the original DataFrame
    keyword_df = df[df['Assigned Topic'].str.lower() == keyword.lower()]
    
    # Store the filtered DataFrame in the dictionary
    dataframes[f'df_{keyword}'] = keyword_df

# Now, dataframes is a dictionary where each key is the name of the DataFrame (e.g., 'df_quality')
# and the corresponding value is the filtered DataFrame.
# You can access the DataFrames like this:
df_quality = dataframes['df_quality']
df_prices = dataframes['df_prices']
df_overall = dataframes['df_overall']
df_ambiance = dataframes['df_ambiance']
df_service = dataframes['df_service']


In [ ]:
df_service

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import Counter

# Create a dictionary to associate DataFrame names with actual DataFrames
dataframe_dict = {
    'df_quality': df_quality,
    'df_prices': df_prices,
    'df_overall': df_overall,
    'df_ambiance': df_ambiance,
    'df_service': df_service
}

# Create a function to preprocess and analyze the reviews
def identify_keywords(review_text):
    words = word_tokenize(review_text.lower())
    words = [word for word in words if word is not None and word.isalpha() and word not in stopwords.words('english') and word not in ["get", "back", "really"]]
    word_count = Counter(words)
    return word_count

# Function to extract nouns based on part-of-speech tagging
def extract_nouns(word_list):
    tagged_words = pos_tag(word_list)
    nouns = [word for word, tag in tagged_words if tag in ['NN', 'NNS', 'NNP', 'NNPS']]
    return nouns

# Create variables to store the most common nouns for each DataFrame
most_common_nouns_df_quality = []
most_common_nouns_df_prices = []
most_common_nouns_df_overall = []
most_common_nouns_df_ambiance = []
most_common_nouns_df_service = []

# Iterate through each DataFrame in the dictionary
for df_name, df in dataframe_dict.items():
    keywords = []

    for index, row in df.iterrows():
        review_text = row['Reviews']
        word_count = identify_keywords(review_text)

        # Append the words to the list of keywords
        keywords.extend(word_count.keys())

    # Extract nouns based on part-of-speech tagging and remove common stopwords
    keywords = extract_nouns(keywords)

    # Count the frequency of each noun across the DataFrame
    keyword_counts = Counter(keywords)

    # Store the top 10 nouns in the corresponding variable
    if df_name == 'df_quality':
        most_common_nouns_df_quality = [word for word, count in keyword_counts.most_common(50)]
    elif df_name == 'df_prices':
        most_common_nouns_df_prices = [word for word, count in keyword_counts.most_common(50)]
    elif df_name == 'df_overall':
        most_common_nouns_df_overall = [word for word, count in keyword_counts.most_common(50)]
    elif df_name == 'df_ambiance':
        most_common_nouns_df_ambiance = [word for word, count in keyword_counts.most_common(50)]
    elif df_name == 'df_service':
        most_common_nouns_df_service = [word for word, count in keyword_counts.most_common(50)]

# Now, you can access the lists as variables, e.g., most_common_nouns_df_quality, most_common_nouns_df_prices, etc.


In [ ]:
all_lists = [most_common_nouns_df_quality, most_common_nouns_df_prices, most_common_nouns_df_overall, most_common_nouns_df_ambiance, most_common_nouns_df_service]
# Create a set to store common words
common_words = set()

# Iterate through all pairs of lists to find common words
for i in range(len(all_lists)):
    for j in range(i + 1, len(all_lists)):
        common_words |= set(all_lists[i]) & set(all_lists[j])

# Remove common words from each list
for i in range(len(all_lists)):
    all_lists[i] = [word for word in all_lists[i] if word not in common_words]


In [ ]:
most_common_nouns_df_quality=all_lists[0]
most_common_nouns_df_prices=all_lists[1]
most_common_nouns_df_overall=all_lists[2]
most_common_nouns_df_ambiance=all_lists[3]
most_common_nouns_df_service=all_lists[4]

In [ ]:
qual_values_to_remove = ['lot', 'bit','home','street']
most_common_nouns_df_quality = [x for x in most_common_nouns_df_quality if x not in qual_values_to_remove]

In [ ]:
overall_values_to_remove = ['house', 'nothing','kind','beach','thing','disappoint']
most_common_nouns_df_overall = [x for x in most_common_nouns_df_overall if x not in overall_values_to_remove]

In [ ]:
amb_values_to_remove = ['meals', 'carbonara','carbo','lasagna','ones','minestrone','cajun','things','daughter','hunger','one']
most_common_nouns_df_ambiance = [x for x in most_common_nouns_df_ambiance if x not in amb_values_to_remove]

In [ ]:
most_common_nouns_df_service
serv_values_to_remove = ['part', 'way','weekend','try','someone','lunch']
most_common_nouns_df_service = [x for x in most_common_nouns_df_service if x not in serv_values_to_remove]

In [ ]:
#Append back to predefined topics

In [ ]:
# Create a dictionary to store the results
predefined_topics = predefined_topics.copy()  # Copy the predefined_topics

# Iterate through your lists and append words to the corresponding predefined_topics keys
for keyword, word_list in [("service", most_common_nouns_df_service), ("quality", most_common_nouns_df_quality), ("ambiance", most_common_nouns_df_ambiance), ("prices", most_common_nouns_df_prices)]:
    for word in word_list:
        if word not in predefined_topics[keyword]:
            predefined_topics[keyword].append(word)


In [ ]:
#Rerun the model
#word count x sentiment x similarity - no negative keyword emphasis - 1 (apply to negative keyword emphasis)
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from textblob import TextBlob  # For sentiment analysis
from sklearn.metrics.pairwise import cosine_similarity


# Preprocess the reviews
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum()]  # Remove non-alphanumeric characters
    return tokens

reviews=df_copy['processed_reviews'].to_list()
processed_reviews = [preprocess_text(review) for review in reviews]

# Train Word2Vec model on the preprocessed reviews
word2vec_model = Word2Vec(processed_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Negative sentiment words
negative_words = [
    "bad", "horrible", "terrible", "poor", "disgusting",
    "awful", "unpleasant", "dreadful", "unappetizing",
    "disappointing", "mediocre", "unpalatable", "inferior",
    "overpriced", "unsatisfactory", "disastrous", "unimpressive",
    "bland", "gross", "repulsive", "underwhelming",
    "inedible", "unflavorful", "miserable", "regrettable",
    "subpar", "lousy", "offensive", "unenjoyable",
    "stale", "inedible", "undercooked", "soggy","horrendous"]

negative_word_weights = {
    "bad": 1.0,
    "horrible": 2.0,
    "Horrendous":2.0,
    "terrible": 2.0,
    "poor": 1.0,
    "disgusting": 2.0,
    "awful": 1.2,
    "unpleasant": 1.3,
    "dreadful": 1.5,
    "unappetizing": 1.2,
    "disappointing": 1.2,
    "mediocre": 1.1,
    "unpalatable": 1.2,
    "inferior": 1.0,
    "overpriced": 1.2,
    "unsatisfactory": 1.1,
    "disastrous": 1.5,
    "unimpressive": 1.1,
    "bland": 1.0,
    "gross": 1.2,
    "repulsive": 1.4,
    "underwhelming": 1.1,
    "inedible": 1.3,
    "unflavorful": 1.2,
    "miserable": 1.3,
    "regrettable": 1.2,
    "subpar": 1.1,
    "lousy": 1.2,
    "offensive": 1.3,
    "unenjoyable": 1.2,
    "stale": 1.1,
    "undercooked": 1.2,
    "soggy": 1.2,
}

# Transition words for sentence transitions
sentence_transitions = ['but', 'and', 'however', 'although', 'yet','so', 'nevertheless', 'while', 'though', 'or']


def calculate_sentiment(review):
    sentiment_scores = [TextBlob(word).sentiment.polarity for word in review]
    sentiment_distribution = [score - 1 for score in sentiment_scores]  # Shift to negative range for negative sentiment
    sentiment_distribution /= np.sum(np.abs(sentiment_distribution))  # Normalize
    sentiment = np.argmax(sentiment_distribution)

    # Adjust the sentiment score based on the order of the words and the context
    for i in range(len(sentiment_scores)):
        if sentiment_scores[i] > 0:
            sentiment_distribution[i] *= (-1) ** (i + 1)

    sentiment = np.argmax(sentiment_distribution)

    return sentiment

def calculate_sentiment_entropy(review):
    sentiment_scores = [TextBlob(word).sentiment.polarity for word in review]
    entropy = -sum(score * np.log2(score) for score in sentiment_scores if score != 0)
    return entropy

def get_sentiment(word):
    return TextBlob(word).sentiment.polarity

def get_main_topic(review):
    review_text = ' '.join(review)  # Convert preprocessed words back into a string
    sentiment_scores = [get_sentiment(word) for word in review]  # Define sentiment_scores here
    sentiment = calculate_sentiment(review_text)

    negative_sentiment_words = [word for word in review if word in negative_words]
    negative_sentiment_scores = [get_sentiment(word) for word in negative_sentiment_words]

    review_embedding = np.nanmean(
        [word2vec_model.wv[word] for word in review if word in word2vec_model.wv.key_to_index], axis=0
    )

    if np.any(np.isnan(review_embedding)):
        return None

    max_score = float('-inf')
    main_topic = None
    second_max_score = float('-inf')
    second_topic = None
    highest_negative_score = 0
    prioritized_categories = []

    for topic, words in predefined_topics.items():
        count = sum(1 for word in words if word in review)

        topic_embedding = np.nanmean(
            [word2vec_model.wv[word] for word in words if word in word2vec_model.wv.key_to_index], axis=0
        )

        if np.any(np.isnan(topic_embedding)):
            category_score = "N/A"
        else:
            similarity = cosine_similarity([review_embedding], [topic_embedding])[0][0]

            # Prioritize topics containing negative sentiment keywords
            negative_sentiment_score = sum(1 for word in review if word in negative_words)
            if negative_sentiment_score > 0 and any(word in words for word in negative_words):
                combined_score = similarity + (0.5 * negative_sentiment_score) + (0.5 * count)
            else:
                combined_score = similarity + (0.5 * sentiment) + (0.5 * count)

            if combined_score > max_score:
                # Update the second topic
                second_max_score = max_score
                second_topic = main_topic
                max_score = combined_score
                main_topic = topic
                prioritized_categories = [topic]
            elif combined_score == max_score:
                # Update the second topic if the score is the same
                second_max_score = max(second_max_score, combined_score)
                second_topic = topic

    sentiment_entropy = calculate_sentiment_entropy(review)
    if sentiment_entropy > 1.0:
        return f"{main_topic}, {second_topic}" if second_topic else main_topic
    else:
        return f"{main_topic}, {second_topic}" if second_topic else main_topic



# Assign each review to one of the predefined topics
review_topics = [get_main_topic(review) for review in processed_reviews]

# Print the assigned topics for each review along with scores and negative word information
for i, review in enumerate(reviews):
    main_topic = review_topics[i]
    count_negative_words = sum(1 for word in preprocess_text(review) if word in negative_words)
    negative_word_score = sum(negative_word_weights.get(word, 1.0) for word in preprocess_text(review) if word in negative_words)
    
    review_words = preprocess_text(review)
    review_embedding = np.nanmean(
        [word2vec_model.wv[word] for word in review_words if word in word2vec_model.wv.key_to_index], axis=0
    )
    
    sentiment_scores = [TextBlob(word).sentiment.polarity for word in review_words]
    sentiment = np.argmax([score + 1 for score in sentiment_scores])  # Shift to positive range
    review_data = {
        'Assigned Topic': main_topic,
        'Count of Negative Keywords': count_negative_words,
        'Aggregate Negative Score': negative_word_score
    }
    for category, words in predefined_topics.items():
        topic_embedding = np.nanmean(
            [word2vec_model.wv[word] for word in words if word in word2vec_model.wv.key_to_index], axis=0
        )
        
        if np.any(np.isnan(topic_embedding)):
            category_score = "N/A"
        else:
            similarity = cosine_similarity([review_embedding], [topic_embedding])[0][0]
            sentiment_score = (0.5 * sentiment) if any(word in words for word in review_words) else 0
            category_score = similarity + sentiment_score
            if category == 'service':
                review_data['Service Score'] = category_score
            elif category == 'ambiance':
                review_data['Ambiance Score'] = category_score
            elif category == 'prices':
                review_data['Prices Score'] = category_score
            elif category == 'quality':
                review_data['Quality Score'] = category_score
            elif category == 'overall':
                review_data['Overall Score'] = category_score
    #review_data['Overall Score'] = max(
        #review_data.get('Overall Score', float('-inf')),
        #review_data.get('Service Score', float('-inf')),
        #review_data.get('Ambiance Score', float('-inf')),
        #review_data.get('Prices Score', float('-inf')),
        #review_data.get('Quality Score', float('-inf'))
        #)
    df_copy.loc[df_copy.index[i], review_data.keys()] = list(review_data.values())


In [ ]:
df_copy

# Example Review Analysis - Jess S.